# Importing libraries

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re

import emoji
from googletrans import Translator

from transformers import pipeline

from sentence_transformers import CrossEncoder

from urllib.parse import urlparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from sklearn.preprocessing import StandardScaler
from datasets import Dataset

# Loading the dataset

scraped data from apify google maps review scraper as well as google maps scraper 

In [219]:
place_data = pd.read_csv('dataset/places_data.csv')
place_data = place_data.drop_duplicates()
place_data

,title,totalScore,reviewsCount,street,city,state,countryCode,website,phone,categoryName,url,place_id
0,Tongue Tip Lanzhou Beef Noodle 舌尖尖 (Downtown E...,4.7,308,1 Pasir Ris Close E!Avenue@Downtown East #02-322,Singapore,NaN,SG,https://www.facebook.com/Tonguetipbeefnoodles/,+65 9780 8123,Halal restaurant,https://www.google.com/maps/search/?api=1&quer...,ChIJ3YGBcSI92jER3UaKr6eQCjI
1,Lemongrass,4.4,1078,"476 Upper Serangoon View, #02-06",Singapore,NaN,SG,https://qashiereats.com/Lemongrass,+65 8068 6965,Thai restaurant,https://www.google.com/maps/search/?api=1&quer...,ChIJ6bdDIk0X2jERzZHtC7S6ceE
2,Lau Wang Claypot Delights @ Punggol Oasis Terr...,4.6,1518,"681 Punggol Dr., #01-15",Singapore,NaN,SG,https://www.lauwangclaypot.com/,NaN,Restaurant,https://www.google.com/maps/search/?api=1&quer...,ChIJY5b4lRgV2jERkw16w9mMBQ4
3,Sanook Kitchen (Waterway Point),4.6,925,"83 Punggol Central, #B1-13 Waterway Point West...",Singapore,NaN,SG,https://sanookkitchen.com.sg/,+65 6231 1597,Thai restaurant,https://www.google.com/maps/search/?api=1&quer...,ChIJJQep-owV2jERrkjXxwYWNJU
4,Ajumma's (Waterway Point),4.8,3451,"02-26 West Wing Waterway, Punggol Central, Poi...",Singapore,NaN,SG,http://www.ajummassg.com/,+65 8749 7993,Korean restaurant,https://www.google.com/maps/search/?api=1&quer...,ChIJ0y_h3jsV2jER0X7-hrLJll8
...,...,...,...,...,...,...,...,...,...,...,...,...
121,National Cancer Centre Singapore,3.6,135,30 Hospital Boulevard,Singapore,NaN,SG,https://www.nccs.com.sg/,+65 6436 8000,Medical Center,https://www.google.com/maps/search/?api=1&quer...,ChIJLQl1k5AZ2jERhIQGGiQQ1Bw
122,Anytime Fitness,4.6,103,"9 Tampines Street 32, #02-01 Tampines Mart",Singapore,NaN,SG,https://www.anytimefitness.sg/gyms/sg-0025/sin...,+65 9017 3533,Gym,https://www.google.com/maps/search/?api=1&quer...,ChIJt3FaVB092jERTrNzmlYINmo
123,Anytime Fitness Changi,4.8,92,"5 Changi Business Park Central 1, #02-01",Singapore,NaN,SG,https://www.anytimefitness.sg/gyms/sg-0130/upp...,+65 8686 6181,Gym,https://www.google.com/maps/search/?api=1&quer...,ChIJvytALvc92jERoTouSnFv5Ws
124,Sembawang Shopping Centre,4.0,5252,604 Sembawang Rd,Singapore,NaN,SG,http://www.sembawangsc.com.sg/,+65 6757 8000,Shopping mall,https://www.google.com/maps/search/?api=1&quer...,ChIJ5-zrBoEU2jERhLnuZ0bgvUw


In [220]:
review_data = pd.read_csv('dataset/all_reviews.csv')
review_data= review_data.drop_duplicates()
review_data

,title,url,stars,name,reviewUrl,text
0,Anytime Fitness,https://www.google.com/maps/search/?api=1&quer...,4,Than pohwai,https://www.google.com/maps/reviews/data=!4m8!...,NaN
1,Anytime Fitness,https://www.google.com/maps/search/?api=1&quer...,1,Rah,https://www.google.com/maps/reviews/data=!4m8!...,Management in this region is not the best! Not...
2,Anytime Fitness,https://www.google.com/maps/search/?api=1&quer...,5,Lin HaiHong,https://www.google.com/maps/reviews/data=!4m8!...,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...
3,Anytime Fitness,https://www.google.com/maps/search/?api=1&quer...,5,Zentov,https://www.google.com/maps/reviews/data=!4m8!...,"I signed up today, and firdaus made the proces..."
4,Anytime Fitness,https://www.google.com/maps/search/?api=1&quer...,5,Tyler,https://www.google.com/maps/reviews/data=!4m8!...,Firdaus has been very patient in assisting me ...
...,...,...,...,...,...,...
3795,Octobox,https://www.google.com/maps/search/?api=1&quer...,5,Patrick Dirks,https://www.google.com/maps/reviews/data=!4m8!...,NaN
3796,Octobox,https://www.google.com/maps/search/?api=1&quer...,5,Lim Jin Yin,https://www.google.com/maps/reviews/data=!4m8!...,NaN
3797,Octobox,https://www.google.com/maps/search/?api=1&quer...,5,a,https://www.google.com/maps/reviews/data=!4m8!...,Amaze
3798,Octobox,https://www.google.com/maps/search/?api=1&quer...,5,marcus tang,https://www.google.com/maps/reviews/data=!4m8!...,amazing


In [ ]:
label_data = pd.read_csv('dataset/label_data.csv')
label_data

In [ ]:
place_data["place_id"] = place_data["url"].apply(lambda x: re.search(r"place_id=([^&]+)", x).group(1) if pd.notnull(x) and re.search(r"place_id=([^&]+)", x) else None)
review_data["place_id"] = review_data["url"].apply(lambda x: re.search(r"place_id=([^&]+)", x).group(1) if pd.notnull(x) and re.search(r"place_id=([^&]+)", x) else None)

merge_1 = pd.merge(review_data[['title', 'stars', 'name', 'text', 'place_id', 'reviewUrl']], 
                   place_data[['totalScore', 'reviewsCount', 'street', 'website', 'phone', 'countryCode', 'categoryName', 'place_id']], 
                   on='place_id', how='left')

df = pd.merge(merge_1, label_data, on="reviewUrl", how='left')

df = df.rename(columns={'title': 'Business Name', 'stars': 'Review', 'name': 'Reviewer Name', 'text': 'Review Text', 
                        'totalScore': 'Business Rating', 'reviewsCount': 'Total Reviews', 'street': 'Address', 
                        'countryCode': 'Country Code', 'categoryName': 'Category'})

df = df.drop_duplicates()
df

,Business Name,Review,Reviewer Name,Review Text,place_id,Business Rating,Total Reviews,Address,website,phone,Country Code,Category
0,Anytime Fitness,4,Than pohwai,NaN,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym
1,Anytime Fitness,1,Rah,Management in this region is not the best! Not...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym
2,Anytime Fitness,5,Lin HaiHong,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym
3,Anytime Fitness,5,Zentov,"I signed up today, and firdaus made the proces...",ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym
4,Anytime Fitness,5,Tyler,Firdaus has been very patient in assisting me ...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym
...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Octobox,5,Patrick Dirks,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store
3792,Octobox,5,Lim Jin Yin,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store
3793,Octobox,5,a,Amaze,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store
3794,Octobox,5,marcus tang,amazing,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store


# Data cleaning

## 1. Check if review has no text

In [222]:
df["empty_review"] = df["Review Text"].apply(lambda x: 1 if pd.isnull(x) or x.strip() == "" else 0)

## 2. Remove HTML tags and emojis

In [ ]:
def remove_html_and_emojis(text):
    if pd.isna(text):
        return np.nan
    
    # Remove emojis (all Unicode emojis, no need to update ranges)
    text = emoji.replace_emoji(text, replace="")  
    
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)  
    
    return text.strip()

df["Review Text"] = df["Review Text"].apply(remove_html_and_emojis)
df

## 3. Translation

In [ ]:
translator = Translator()

def detect_language(text):
    if pd.isnull(text) or text.strip() == "":
        return None
    try:
        result = translator.detect(text)
        return result.lang
    except Exception:
        return None

def translate_to_english(text):
    if pd.isnull(text) or text.strip() == "":
        return None
    try:
        result = translator.translate(text, dest="en")
        return result.text
    except Exception as e:
        return f"Error: {e}"

# Detect language
df["og_lang_code"] = df["Review Text"].apply(detect_language)

# Force translate everything to English
df["Review Text Translated"] = df["Review Text"].apply(translate_to_english)

df

,Business Name,Review,Reviewer Name,Review Text,place_id,Business Rating,Total Reviews,Address,website,phone,Country Code,Category,empty_review,lang_code,Review Text Translated,is_translated
0,Anytime Fitness,4,Than pohwai,NaN,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,1,None,None,0
1,Anytime Fitness,1,Rah,Management in this region is not the best! Not...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Management in this region is not the best! Not...,0
2,Anytime Fitness,5,Lin HaiHong,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,zh-CN,Thank you Desmond for saving a woman who faint...,1
3,Anytime Fitness,5,Zentov,"I signed up today, and firdaus made the proces...",ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,"I signed up today, and firdaus made the proces...",0
4,Anytime Fitness,5,Tyler,Firdaus has been very patient in assisting me ...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Firdaus has been very patient in assisting me ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Octobox,5,Patrick Dirks,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0
3792,Octobox,5,Lim Jin Yin,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0
3793,Octobox,5,a,Amaze,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,Amaze,0
3794,Octobox,5,marcus tang,amazing,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,amazing,0


In [ ]:
translation_error = df[df["Review Text Translated"].fillna("").str.contains("Error:")]

while len(translation_error) > 0:
    error_mask = df["Review Text Translated"].fillna("").str.contains("Error:")
    df.loc[error_mask, "Review Text Translated"] = df.loc[error_mask].apply(
        lambda row: translate_to_english(row["Review Text"]), axis=1
    )
    translation_error = df[df["Review Text Translated"].fillna("").str.contains("Error:")]

df

,Business Name,Review,Reviewer Name,Review Text,place_id,Business Rating,Total Reviews,Address,website,phone,Country Code,Category,empty_review,lang_code,Review Text Translated,is_translated
0,Anytime Fitness,4,Than pohwai,NaN,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,1,None,None,0
1,Anytime Fitness,1,Rah,Management in this region is not the best! Not...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Management in this region is not the best! Not...,0
2,Anytime Fitness,5,Lin HaiHong,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,zh-CN,Thank you Desmond for saving a woman who faint...,1
3,Anytime Fitness,5,Zentov,"I signed up today, and firdaus made the proces...",ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,"I signed up today, and firdaus made the proces...",0
4,Anytime Fitness,5,Tyler,Firdaus has been very patient in assisting me ...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Firdaus has been very patient in assisting me ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Octobox,5,Patrick Dirks,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0
3792,Octobox,5,Lim Jin Yin,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0
3793,Octobox,5,a,Amaze,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,Amaze,0
3794,Octobox,5,marcus tang,amazing,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,amazing,0


## 4. Group categories

In [226]:
CATEGORY_MAPPING = {
    "Gym": "Gym",
    "Sporting goods store": "Sporting goods store",
    "Sports complex": "Sports complex",
    "Community center": "Community center",
    "Business center": "Business center",
    "Shopping mall": "Shopping mall",
    "Recycling center": "Recycling center",
    
    # Restaurants (grouped)
    "Restaurant": "Restaurant",
    "Halal restaurant": "Restaurant",
    "Thai restaurant": "Restaurant",
    "Chinese restaurant": "Restaurant",
    "Korean restaurant": "Restaurant",
    "Ramen restaurant": "Restaurant",
    "Seafood restaurant": "Restaurant",
    "Korean barbecue restaurant": "Restaurant",
    "Indian restaurant": "Restaurant",
    "Hawker stall": "Restaurant",
    "Japanese restaurant": "Restaurant",
    "Western restaurant": "Restaurant",
    "Sushi restaurant": "Restaurant",
    "Hong Kong style fast food restaurant": "Restaurant",
    "Cafe": "Restaurant",
    
    # Clothing (grouped)
    "Clothing store": "Clothing store",
    "Plus size clothing store": "Clothing store",
    "Women's clothing store": "Clothing store",
    
    # Computer (grouped)
    "Computer store": "Computer store",
    "Computer service": "Computer store",
    "Computer repair service": "Computer store",
    
    # Supermarket / Grocery (grouped)
    "Supermarket": "Supermarket",
    "Grocery store": "Supermarket",
    "Indian grocery store": "Supermarket",
    "Convenience store": "Supermarket",
    
    # Medical / Healthcare (grouped)
    "Medical clinic": "Medical clinic",
    "Dental clinic": "Medical clinic",
    "Skin care clinic": "Medical clinic",
    "Walk-in clinic": "Medical clinic",
    "General practitioner": "Medical clinic",
    "Medical Center": "Medical clinic",
    "Hospital": "Medical clinic",
    "Community health center": "Medical clinic",
    "Chinese medicine clinic": "Medical clinic",
}

In [227]:
df["mapped_category"] = df["Category"].map(CATEGORY_MAPPING)
df

,Business Name,Review,Reviewer Name,Review Text,place_id,Business Rating,Total Reviews,Address,website,phone,Country Code,Category,empty_review,lang_code,Review Text Translated,is_translated,mapped_category
0,Anytime Fitness,4,Than pohwai,NaN,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,1,None,None,0,Gym
1,Anytime Fitness,1,Rah,Management in this region is not the best! Not...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Management in this region is not the best! Not...,0,Gym
2,Anytime Fitness,5,Lin HaiHong,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,zh-CN,Thank you Desmond for saving a woman who faint...,1,Gym
3,Anytime Fitness,5,Zentov,"I signed up today, and firdaus made the proces...",ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,"I signed up today, and firdaus made the proces...",0,Gym
4,Anytime Fitness,5,Tyler,Firdaus has been very patient in assisting me ...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Firdaus has been very patient in assisting me ...,0,Gym
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Octobox,5,Patrick Dirks,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0,Supermarket
3792,Octobox,5,Lim Jin Yin,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0,Supermarket
3793,Octobox,5,a,Amaze,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,Amaze,0,Supermarket
3794,Octobox,5,marcus tang,amazing,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,amazing,0,Supermarket


# Feature Engineering

## 1. Length of review

In [228]:
df["review_length"] = df["Review Text Translated"].apply(
    lambda x: len(str(x).split()) if pd.notna(x) and str(x).strip() != "" else 0)
df

,Business Name,Review,Reviewer Name,Review Text,place_id,Business Rating,Total Reviews,Address,website,phone,Country Code,Category,empty_review,lang_code,Review Text Translated,is_translated,mapped_category,review_length
0,Anytime Fitness,4,Than pohwai,NaN,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,1,None,None,0,Gym,0
1,Anytime Fitness,1,Rah,Management in this region is not the best! Not...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Management in this region is not the best! Not...,0,Gym,86
2,Anytime Fitness,5,Lin HaiHong,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,zh-CN,Thank you Desmond for saving a woman who faint...,1,Gym,40
3,Anytime Fitness,5,Zentov,"I signed up today, and firdaus made the proces...",ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,"I signed up today, and firdaus made the proces...",0,Gym,29
4,Anytime Fitness,5,Tyler,Firdaus has been very patient in assisting me ...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Firdaus has been very patient in assisting me ...,0,Gym,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Octobox,5,Patrick Dirks,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0,Supermarket,0
3792,Octobox,5,Lim Jin Yin,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0,Supermarket,0
3793,Octobox,5,a,Amaze,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,Amaze,0,Supermarket,1
3794,Octobox,5,marcus tang,amazing,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,amazing,0,Supermarket,1


## 2. Sentiment score

In [197]:
# sentiment_pipeline = pipeline(
#     "sentiment-analysis",
#     model="distilbert-base-uncased-finetuned-sst-2-english"
# )

# def get_sentiment_chunked(text, pipeline, max_words=500):
#     """
#     Splits long reviews into chunks, runs sentiment analysis on each,
#     and averages the scores.
#     """
#     if pd.isna(text) or str(text).strip() == "":
#         return 0.0
    
#     words = str(text).split()
#     chunks = [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]
    
#     scores = []
#     for chunk in chunks:
#         result = pipeline(chunk, truncation=True)[0]
#         label = result["label"]
#         score = result["score"]
#         score = score if label == "POSITIVE" else -score
#         scores.append(score)
    
#     return sum(scores) / len(scores)

# df["sentiment_score"] = df["Review Text Translated"].apply(
#     lambda x: get_sentiment_chunked(x, sentiment_pipeline)
# )

# df

## 3. Relevence between reviews and keywords 

In [229]:
CATEGORY_KEYWORDS = {
    "Gym": [
        "gym", "workout", "exercise", "fitness", "weights", "treadmill", 
        "trainer", "membership", "equipment", "cleanliness", "staff", "machines",
        "coach", "classes", "member", "manager", "personal"
    ],
    
    "Sporting goods store": [
        "sports", "equipment", "gear", "shoes", "jersey", "ball", "racket", 
        "fitness", "outdoor", "indoor", "brands", "price", "staff", "manager"
    ],
    
    "Sports complex": [
        "sports", "stadium", "court", "field", "swimming", "track", "facilities", 
        "training", "events", "fitness", "exercise", "cleanliness", "staff", "active"
    ],
    
    "Community center": [
        "community", "activities", "events", "classes", "programs", "facilities", 
        "support", "volunteer", "meeting", "youth", "elderly", "services", "cc", "food",
        "hawker", "toilet", "recreational", "active"
    ],
    
    "Business center": [
        "business", "office", "workspace", "meeting", "conference", "facilities", 
        "services", "support", "location", "professional", "staff"
    ],
    
    "Shopping mall": [
        "mall", "shops", "stores", "brands", "food court", "parking", "cinema", 
        "crowd", "cleanliness", "facilities", "location", "security", "toilet"
    ],
    
    "Recycling center": [
        "recycling", "waste", "bins", "plastic", "paper", "metal", "glass", 
        "environment", "collection", "sorting", "facility", "drop-off"
    ],
    
    "Restaurant": [
        "food", "meal", "dishes", "menu", "taste", "flavor", "portion", "price", 
        "service", "waiter", "chef", "ambience", "cleanliness", "toilet", "cutlery", "staff"
    ],
    
    "Clothing store": [
        "clothes", "fashion", "dress", "shirt", "pants", "jeans", "style", 
        "design", "fitting room", "size", "quality", "price", "brands", "staff", "dressing room"
    ],
    
    "Computer store": [
        "computer", "laptop", "desktop", "repair", "service", "software", 
        "hardware", "keyboard", "mouse", "monitor", "IT support", "price", "warranty", "staff"
    ],
    
    "Supermarket": [
        "supermarket", "grocery", "store", "cashier", "checkout", "queue", 
        "product", "price", "discount", "fresh", "vegetables", "fruits", "meat", 
        "cleanliness", "staff", "snacks", "food"
    ],
    
    "Medical clinic": [
        "doctor", "nurse", "clinic", "hospital", "appointment", "treatment", 
        "medicine", "consultation", "check-up", "surgery", "waiting time", 
        "staff", "cleanliness", "pharmacy", "wait", "mc"
    ],
}

In [230]:
cross_encoder = CrossEncoder("cross-encoder/stsb-roberta-base")

def compute_topic_relevance_cross(review_text, mapped_category):
    if not review_text or str(review_text).strip() == "" or pd.isna(review_text):
        return 0
    
    if mapped_category not in CATEGORY_KEYWORDS:
        return 0
    
    keywords = CATEGORY_KEYWORDS[mapped_category]
    
    # Turn keywords into short descriptive phrases
    keyword_phrases = [f"This review is about {kw}" for kw in keywords]
    
    pairs = [(review_text, phrase) for phrase in keyword_phrases]
    scores = cross_encoder.predict(pairs)  # already 0–1
    
    return float(np.max(scores))

# Apply to dataframe
df["topic_relevance"] = df.apply(
    lambda x: compute_topic_relevance_cross(
        x["Review Text Translated"], x["mapped_category"]
    ),
    axis=1,
)

df

,Business Name,Review,Reviewer Name,Review Text,place_id,Business Rating,Total Reviews,Address,website,phone,Country Code,Category,empty_review,lang_code,Review Text Translated,is_translated,mapped_category,review_length,topic_relevance
0,Anytime Fitness,4,Than pohwai,NaN,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,1,None,None,0,Gym,0,0.000000
1,Anytime Fitness,1,Rah,Management in this region is not the best! Not...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Management in this region is not the best! Not...,0,Gym,86,0.442945
2,Anytime Fitness,5,Lin HaiHong,感谢 Desmond 在关键时刻救了一位晕倒的女士，本人真心谢谢见义勇为的你.你是一位好员工...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,zh-CN,Thank you Desmond for saving a woman who faint...,1,Gym,40,0.127851
3,Anytime Fitness,5,Zentov,"I signed up today, and firdaus made the proces...",ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,"I signed up today, and firdaus made the proces...",0,Gym,29,0.150197
4,Anytime Fitness,5,Tyler,Firdaus has been very patient in assisting me ...,ChIJ08t6PjYP2jERi4sIp8Z1Sw0,4.7,215,"638 Jurong West Street 61, #03-01 Pioneer Mall",https://www.anytimefitness.sg/gyms/sg-0095/pio...,+65 8500 4500,SG,Gym,0,en,Firdaus has been very patient in assisting me ...,0,Gym,29,0.435362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Octobox,5,Patrick Dirks,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0,Supermarket,0,0.000000
3792,Octobox,5,Lim Jin Yin,NaN,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,1,None,None,0,Supermarket,0,0.000000
3793,Octobox,5,a,Amaze,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,Amaze,0,Supermarket,1,0.183569
3794,Octobox,5,marcus tang,amazing,ChIJ1ZD3MgAP2jERQvGVLFKfqVQ,4.9,189,"50 Nanyang Ave, Nanyang Technological Universi...",NaN,NaN,SG,Convenience store,0,en,amazing,0,Supermarket,1,0.158901


In [ ]:
df.to_csv("dataset/final_df.csv", index=False)

# Labelling

- 0: Low Quality Review 
- 1: High Quality Review 
- 2: Fake Review
- 3: Irrelevant Review 
- 4: Advertisement 

# Model Training

loading dataset

In [ ]:
def load_dataset(df):
    texts = df["Review Text Translated"].tolist()
    labels = df["review_label"].tolist()

    # Select numeric features
    features = df[["place_id", "Business Rating", "Total Reviews", 
                   "mapped_category", "Review", "review_length", "topic_relevance"]].fillna(0).values

    # Normalize features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    return texts, features, labels, scaler

tokenize texts

In [ ]:
def tokenize_texts(texts, tokenizer, max_len=128):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

building model

In [ ]:
def build_model(model_name, num_features, num_labels):
    text_encoder = AutoModel.from_pretrained(model_name)

    # Classifier head (text + features)
    classifier = nn.Sequential(
        nn.Linear(text_encoder.config.hidden_size + num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, num_labels)
    )

    return text_encoder, classifier

def forward_pass(text_encoder, classifier, input_ids, attention_mask, features, labels=None):
    outputs = text_encoder(input_ids=input_ids, attention_mask=attention_mask)
    pooled_output = outputs.last_hidden_state[:, 0] 

    combined = torch.cat([pooled_output, features], dim=1)

    logits = classifier(combined)

    loss = None
    if labels is not None:
        loss = F.cross_entropy(logits, labels)

    return {"loss": loss, "logits": logits}

train model

In [ ]:
def train_model(texts, features, labels, model_name="microsoft/deberta-v3-base"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    text_encodings = tokenize_texts(texts, tokenizer)

    dataset = Dataset.from_dict({
        "input_ids": text_encodings["input_ids"],
        "attention_mask": text_encodings["attention_mask"],
        "features": torch.tensor(features, dtype=torch.float32),
        "labels": torch.tensor(labels, dtype=torch.long)
    })

    text_encoder, classifier = build_model(model_name, features.shape[1], num_labels=5)

    def model_fn(batch):
        return forward_pass(
            text_encoder,
            classifier,
            batch["input_ids"],
            batch["attention_mask"],
            batch["features"],
            batch["labels"]
        )

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=50
    )

    trainer = Trainer(
        model=model_fn,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=dataset
    )

    trainer.train()
    return trainer, tokenizer

In [ ]:
texts, features, labels, scaler = load_dataset(non_empty_text_df)
trainer, tokenizer = train_model(texts, features, labels)

In [ ]:
# def business_in_url(url: str, business_name: str) -> bool:
#     parsed_url = urlparse(url)
#     url_text = (parsed_url.netloc + parsed_url.path).lower()
    
#     cleaned_name = re.sub(r"[^a-z0-9\s]", "", business_name.lower())
#     words = cleaned_name.split()
    
#     return any(word in url_text for word in words if len(word) > 2)

# # Function to classify review text
# def classify_url(row):
#     text = row["Review Text Translated"]
#     business_name = row["Business Name"]

#     if pd.isnull(text) or str(text).strip() == "":
#         return 0
    
#     text = text.strip()
#     if (text.startswith("www.") or 
#         text.startswith("https") or 
#         text.endswith(".com")):
#         if not business_in_url(text, business_name):
#             return 4

# # Apply to dataframe
# df["review_label"] = df.apply(classify_url, axis=1)

# df
